In [ ]:
# This makes plots showing the effective temperature retrievals based on synthetic spectra 
# produced by R.W.

# Created from parent restacking_scraped_data.ipynb 2021 March 17 by E.S.

In [1]:
import pandas as pd
#from astropy.io import fits
from astropy.io.fits import getdata
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

%matplotlib qt

In [2]:
# name of csv file with EWs as produced by pipeline
ew_good_data_poststack_file_name = "/Users/bandari/Documents/git.repos/rrlyrae_metallicity/rrlyrae_metallicity/ew_products/20210225_restacked_ew_info_good_only.csv"

# read in
df_poststack = pd.read_csv(ew_good_data_poststack_file_name)

In [3]:
def line_fit(x_data_pass, y_data_pass):

    # remove the stuff outside of 6000-7250 K
    #x_data_rrl = x_data_pass.where(np.logical_and(x_data_pass>=5900,x_data_pass<=7350))
    #y_data_rrl = x_data_pass.where(np.logical_and(x_data_pass>=5900,x_data_pass<=7350))
    x_data_rrl = x_data_pass[np.where(np.logical_and(y_data_pass>=5900,y_data_pass<=7350))]
    y_data_rrl = y_data_pass[np.where(np.logical_and(y_data_pass>=5900,y_data_pass<=7350))]
    
    coeff, cov = np.polyfit(x_data_rrl, y_data_rrl, 1, full=False, cov=True)
    m = coeff[0]
    b = coeff[1]
    err_m = np.sqrt(np.diag(cov))[0]
    err_b = np.sqrt(np.diag(cov))[1]

    print("---------")
    print("Note stuff outside of 6000-7350 K is not being considered")
    print("m:")
    print(m)
    print("err_m:")
    print(err_m)
    print("b:")
    print(b)
    print("err_b:")
    print(err_b)
    
    return m, b

In [4]:
# plot: how do Balmer lines scale with Teff?

plt.clf()
plt.title("Scaling of lines with Hdelta")
plt.scatter(df_poststack["Teff"],df_poststack["EW_Hbeta"], s=3, label="Hbeta")
plt.scatter(df_poststack["Teff"],np.add(df_poststack["EW_Hgamma"],6), s=3, label="Hgamma+6")
plt.scatter(df_poststack["Teff"],np.add(df_poststack["EW_Hdelta"],12), s=3, label="Hdel+12")
plt.scatter(df_poststack["Teff"],np.add(df_poststack["EW_Balmer"],18), s=3, label="Net Balmer+18")
plt.scatter(df_poststack["Teff"],np.add(df_poststack["EW_Heps"],24), s=3, label="Heps+24")
#plt.ylim([0,70])
plt.xlabel("Teff (K)")
plt.ylabel("EW (Angstr)")
plt.title("Balmer EW trend with Teff")
plt.legend(ncol=5)
plt.show()
#plt.savefig("junk_balmer_rescalings.pdf")

In [4]:
# find linear trends of {net Balmer, Hdelta, and Hgamma} EW with Teff, entire Teff range

y_data = df_poststack["Teff"].values.astype(float)

# fit a straight line: net Balmer
x_data_Balmer = df_poststack["EW_Balmer"].values.astype(float)
m_Balmer, b_Balmer = line_fit(x_data_Balmer,y_data)
# same, except that [Fe/H] = -3 is neglected
x_data_Balmer_metalrich = df_poststack["EW_Balmer"].where(df_poststack["FeH"] > -2.9).dropna().values.astype(float)
y_data_metalrich = df_poststack["Teff"].where(df_poststack["FeH"] > -2.9).dropna().values.astype(float)
m_Balmer_metalrich, b_Balmer_metalrich = line_fit(x_data_Balmer_metalrich,y_data_metalrich)

# fit a straight line: Hdelta
x_data_Hdelta = df_poststack["EW_Hdelta"].values.astype(float)
m_Hdelta, b_Hdelta = line_fit(x_data_Hdelta,y_data)

# fit a straight line: Hgamma
x_data_Hgamma = df_poststack["EW_Hgamma"].values.astype(float)
m_Hgamma, b_Hgamma = line_fit(x_data_Hgamma,y_data)

---------
Note stuff outside of 6000-7350 K is not being considered
m:
176.890642110337
err_m:
3.8703102046801994
b:
5591.907160176097
err_b:
24.951087080081216
---------
Note stuff outside of 6000-7350 K is not being considered
m:
188.03228016332577
err_m:
2.271224218845659
b:
5491.53408515525
err_b:
15.00176229865291
---------
Note stuff outside of 6000-7350 K is not being considered
m:
185.2279162078319
err_m:
3.202640648339817
b:
5552.614594255033
err_b:
20.38151559802104
---------
Note stuff outside of 6000-7350 K is not being considered
m:
176.04951033311448
err_m:
3.8519065126474787
b:
5523.918371733546
err_b:
26.346320375226036


In [5]:
# calculate retrieved Teff and add new columns to DataFrame to make the plotting easier

df_poststack["Teff_retrieved_Balmer"] = np.add(np.multiply(df_poststack["EW_Balmer"],m_Balmer),b_Balmer)
df_poststack["Teff_retrieved_Hdelta"] = np.add(np.multiply(df_poststack["EW_Hdelta"],m_Hdelta),b_Hdelta)
df_poststack["Teff_retrieved_Hgamma"] = np.add(np.multiply(df_poststack["EW_Hgamma"],m_Hgamma),b_Hgamma)
df_poststack["Teff_retrieved_Balmer_metalrich"] = np.add(np.multiply(df_poststack["EW_Balmer"],m_Balmer_metalrich),b_Balmer_metalrich)

In [6]:
# array of metallicities and logg

feh_values = np.sort(df_poststack["FeH"].drop_duplicates().values)
logg_values = np.sort(df_poststack["logg"].drop_duplicates().values)

In [7]:
# retrieved Balmer values, points colored by Fe/H

plt.clf()

colormap="Reds"
norm = matplotlib.colors.Normalize(vmin=np.min(feh_values),vmax=np.max(feh_values))

f, (a0, a1) = plt.subplots(nrows=2, ncols=1, gridspec_kw={'height_ratios': [2, 1]}, sharex=True)

a0.axvspan(6000, 7250, color='y', alpha=0.5, lw=0,zorder=0) # RRLs in instability strip (Catelan 2015)
a1.axvspan(6000, 7250, color='y', alpha=0.5, lw=0,zorder=0)
a0.plot(df_poststack["Teff"],df_poststack["Teff"],zorder=1,linestyle="--",color="k")
a1.plot([np.min(df_poststack["Teff"]),np.max(df_poststack["Teff"])],[0,0],zorder=1,linestyle="--",color="k")

a0.scatter(df_poststack["Teff"],
            df_poststack["Teff_retrieved_Balmer"],
            c=df_poststack["FeH"], 
            cmap=colormap, norm=norm, edgecolor="k",zorder=2)

a1.scatter(df_poststack["Teff"],
            np.subtract(df_poststack["Teff_retrieved_Balmer_metalrich"],df_poststack["Teff"]),
            c=df_poststack["FeH"], 
            cmap=colormap, norm=norm, edgecolor="k",zorder=2)

'''
# annotation to check the color mapping
for t in range(0,len(df_poststack["FeH"])):
    plt.annotate(str(df_poststack["FeH"][t]), (df_poststack["Teff"][t],df_poststack["Teff_retrieved_Balmer"][t]))
'''
# kludge to add legend while mapping colors correctly
for i in range(0,len(feh_values)):
    # indices reversed to get the order descending in the legend
    a0.scatter([0], [0], cmap=colormap, norm=norm, c=feh_values[-i-1],
                edgecolor="k", label="[Fe/H]="+str(feh_values[-i-1]))
    print(feh_values[i])

a0.set_ylabel("Retrieved T$_{eff}$")
a1.set_xlabel("Injected T$_{eff}$")
a1.set_ylabel("Retrieved T$_{eff}$ - Injected T$_{eff}$\n(based on trend for [Fe/H] $\geq$ -2.5)")
    
f.canvas.draw() # need before legend to render

a0.set_xlim([5500,8000])
a0.set_ylim([5500,8500])

a0.legend(loc="lower right")

plt.show()
#plt.savefig("junk.pdf")

-3.0
-2.5
-2.0
-1.5
-1.0
-0.5
0.0
0.2


In [15]:
scatter_x = np.subtract(50*np.random.rand(len(df_poststack["Teff"])),25)

In [16]:
scatter_x

array([ 15.86111323,  -0.33783677,  -2.35716902, -17.84681728,
        22.8974872 ,  23.84820476,   4.82312207, -22.5012916 ,
        23.74577314, -13.32089523,  18.67231993,  11.99623911,
       -23.79468628,  22.51377906,  21.80881963, -21.76199119,
        -8.021025  ,  20.17784324,  -5.96119137, -16.29699434,
       -21.43640647,   3.67746425, -12.35041912,   2.12907219,
         1.03895967, -11.2375094 ,  -6.72864568,   3.2570502 ,
       -14.78559634,   2.27040838,  11.6908568 ,  23.55346756,
       -12.84894536,  -1.88099164, -13.82947056,   5.59083884,
        19.72977031, -21.75562476, -17.73073208, -19.17982806,
        -1.76635961, -16.47097206, -22.26413403, -12.30115769,
        20.19726035, -18.41945247,  -2.2670442 ,  18.25859257,
        -7.1188626 ,   4.77990536,   4.72451987,   0.44923549,
       -22.62200259,  24.66746619,  15.16074354,  11.97596999,
        10.20494635,  15.13110495,  12.82327669,  15.72348788,
        -4.46641743,  -3.31331459,  18.81019309,  10.35

In [18]:
# retrieved Balmer values, points colored by Fe/H AND sized by logg

plt.clf()

# vector for introducing some scatter in x, to avoid overlapping of data points
scatter_x = np.subtract(60*np.random.rand(len(df_poststack["Teff"])),30)

colormap="Reds"
norm = matplotlib.colors.Normalize(vmin=np.min(feh_values),vmax=np.max(feh_values))

f, (a0, a1) = plt.subplots(nrows=2, ncols=1, gridspec_kw={'height_ratios': [1, 1]}, sharex=True)

a0.axvspan(6000, 7250, color='y', alpha=0.5, lw=0,zorder=0) # RRLs in instability strip (Catelan 2015)
a1.axvspan(6000, 7250, color='y', alpha=0.5, lw=0,zorder=0)
a0.plot(df_poststack["Teff"],df_poststack["Teff"],zorder=1,linestyle="--",color="k")
a1.plot([np.min(df_poststack["Teff"]),np.max(df_poststack["Teff"])],[0,0],zorder=1,linestyle="--",color="k")

a0.scatter(np.add(scatter_x,df_poststack["Teff"]),
            df_poststack["Teff_retrieved_Balmer"],
            c=df_poststack["FeH"], 
            s=np.power(np.divide(df_poststack["logg"],0.7),3),
            cmap=colormap, norm=norm, edgecolor="k",zorder=2)

a1.scatter(np.add(scatter_x,df_poststack["Teff"]),
            np.subtract(df_poststack["Teff_retrieved_Balmer_metalrich"],df_poststack["Teff"]),
            c=df_poststack["FeH"], 
           s=np.power(np.divide(df_poststack["logg"],0.7),3),
            cmap=colormap, norm=norm, edgecolor="k",zorder=2)

'''
# annotation to check the color mapping
for t in range(0,len(df_poststack["FeH"])):
    plt.annotate(str(df_poststack["FeH"][t]), (df_poststack["Teff"][t],df_poststack["Teff_retrieved_Balmer"][t]))
'''
#fehplot = []
# kludge to add legend while mapping colors correctly
for i in range(0,len(feh_values)):
    # indices reversed to get the order descending in the legend
    a0.scatter([0], [0], cmap=colormap, norm=norm, c=feh_values[-i-1],
                edgecolor="k", label="[Fe/H]="+str(feh_values[-i-1]))
    print(feh_values[i])

'''
for i in range(0,len(logg_values)):
    # indices reversed to get the order descending in the legend
    loggpts += a0.scatter([0], [0], c="w", norm=norm,
               s=np.power(np.divide(logg_values[-i-1],0.7),3),
                edgecolor="k")
'''

# logg plot; another kludge
l1 = a0.scatter([0], [0], c="w", norm=norm,
               s=np.power(np.divide(logg_values[-0-1],0.7),3),
                edgecolor="k")
l2 = a0.scatter([0], [0], c="w", norm=norm,
               s=np.power(np.divide(logg_values[-1-1],0.7),3),
                edgecolor="k")
l3 = a0.scatter([0], [0], c="w", norm=norm,
               s=np.power(np.divide(logg_values[-2-1],0.7),3),
                edgecolor="k")
loggpts = []
loggpts.append([l1,l2,l3])
    
f.canvas.draw() # need before legend to render

leg1 = a0.legend(loc='upper left')
# Add second legend for the maxes and mins.
# leg1 will be removed from figure
leg2 = a0.legend(loggpts[0],['log(g)=3.0','log(g)=2.5','log(g)=2.0'], loc='lower right')
# Manually add the first legend back
a0.add_artist(leg1)
    
a0.set_ylabel("Retrieved T$_{eff}$")
a1.set_xlabel("Injected T$_{eff}$")
a1.set_ylabel("Retrieved T$_{eff}$ - Injected T$_{eff}$\n(based on trend within IS for [Fe/H] $\geq$ -2.5)")

a0.set_xlim([5500,8000])
a0.set_ylim([5500,8500])

plt.show()
#plt.savefig("junk.pdf")

-3.0
-2.5
-2.0
-1.5
-1.0
-0.5
0.0
0.2


In [12]:
# retrieved Balmer values, points colored by logg

plt.clf()

colormap="Reds"
norm = matplotlib.colors.Normalize(vmin=np.min(logg_values),vmax=np.max(logg_values))

f, (a0, a1) = plt.subplots(nrows=2, ncols=1, gridspec_kw={'height_ratios': [2, 1]}, sharex=True)

a0.axvspan(6000, 7250, color='y', alpha=0.5, lw=0,zorder=0) # RRLs in instability strip (Catelan 2015)
a1.axvspan(6000, 7250, color='y', alpha=0.5, lw=0,zorder=0)
a0.plot(df_poststack["Teff"],df_poststack["Teff"],zorder=1,linestyle="--",color="k")
a1.plot([np.min(df_poststack["Teff"]),np.max(df_poststack["Teff"])],[0,0],zorder=1,linestyle="--",color="k")

a0.scatter(df_poststack["Teff"],
            df_poststack["Teff_retrieved_Balmer"],
            c=df_poststack["logg"], 
            cmap=colormap, norm=norm, edgecolor="k",zorder=2)

a1.scatter(df_poststack["Teff"],
            np.subtract(df_poststack["Teff_retrieved_Balmer_metalrich"],df_poststack["Teff"]),
            c=df_poststack["logg"], 
            cmap=colormap, norm=norm, edgecolor="k",zorder=2)

'''
# annotation to check the color mapping
for t in range(0,len(df_poststack["FeH"])):
    plt.annotate(str(df_poststack["FeH"][t]), (df_poststack["Teff"][t],df_poststack["Teff_retrieved_Balmer"][t]))
'''
# kludge to add legend while mapping colors correctly
for i in range(0,len(logg_values)):
    # indices reversed to get the order descending in the legend
    a0.scatter([0], [0], cmap=colormap, norm=norm, c=logg_values[-i-1],
                edgecolor="k", label="logg="+str(logg_values[-i-1]))
    print(logg_values[i])

a0.set_ylabel("Retrieved T$_{eff}$")
a1.set_xlabel("Injected T$_{eff}$")
a1.set_ylabel("Retrieved T$_{eff}$ - Injected T$_{eff}$\n(based on trend for [Fe/H] $\geq$ -2.5)")
    
f.canvas.draw() # need before legend to render

a0.set_xlim([5500,8000])
a0.set_ylim([5500,8500])

a0.legend(loc="lower right")

plt.show()
#plt.savefig("junk.pdf")

2.0
2.5
3.0


In [ ]:
# calculate BIC to find best model

In [ ]:
def pred_teff(EW_pass,m_pass,b_pass):
    
    teff_pass = np.add(np.multiply(EW_pass,m_pass),b_pass)
    
    return teff_pass

In [ ]:
Teff_model = pred_teff(df_poststack["EW_Balmer"])